In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/cat_dog
!unzip Archive.zip

In [ ]:
# data
# - train
#   + dogs
#   + cats
# - val
#   + dogs
#   + cats

In [ ]:
# Xay dung model clasify đơn giản
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten, Dense

def build_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD()
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

model = build_model()

In [ ]:
# Load dữ liệu để train model
datagen = ImageDataGenerator(rescale=1.0/255.0)
data_path = "/content/drive/MyDrive/cat_dog/"

train_data = datagen.flow_from_directory(data_path + "train/",
  class_mode='binary', batch_size=64, target_size=(224, 224))
val_data = datagen.flow_from_directory(data_path + "val/",
  class_mode='binary', batch_size=64, target_size=(224, 224))

Found 19906 images belonging to 2 classes.
Found 5094 images belonging to 2 classes.


In [ ]:
# fit model
history = model.fit(train_data, steps_per_epoch=len(train_data),
  validation_data=val_data, validation_steps=len(val_data), epochs=1, verbose=1)


312/312 [==============================] - 156s 461ms/step - loss: 0.6861 - accuracy: 0.5777 - val_loss: 0.6495 - val_accuracy: 0.6078


# Train bang TF.Data

In [ ]:
from imutils import paths
import numpy as np
import os

# 0. Lấy danh sách các file ảnh train
train_image_path = list(paths.list_images(data_path + "train"))
# 1. Lấy danh sách các file ảnh để val
val_image_path = list(paths.list_images(data_path + "val"))
# 2. Tạo danh sách các class: dogs và cats
classNames = np.array(sorted(os.listdir(data_path + "train")))
# 3. Build 1 hàm xử lý ảnh
def preproces_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_png(image, channels=3)
  image = tf.image.resize(image, (224,224))
  image = image / 255.0

  label = tf.strings.split(image_path, os.path.sep)[-2]
  oneHot = label == classNames
  encodedLabel = tf.argmax(oneHot)

  return (image, encodedLabel)


In [ ]:

# 4. Dùng TF.Data để tạo pipeline load và xử lý ảnh
train_dataset = tf.data.Dataset.from_tensor_slices(train_image_path)
train_dataset = (train_dataset
                 .map(preproces_image, num_parallel_calls = tf.data.AUTOTUNE)
                 .batch(64)
                 .prefetch(tf.data.AUTOTUNE)
                 )



In [ ]:

# 4. Dùng TF.Data để tạo pipeline load và xử lý ảnh
val_dataset = tf.data.Dataset.from_tensor_slices(val_image_path)
val_dataset = (val_dataset
                 .map(preproces_image, num_parallel_calls = tf.data.AUTOTUNE)
                 .batch(64)
                 .prefetch(tf.data.AUTOTUNE)
                 )



In [ ]:
# fit model
history = model.fit(train_dataset, steps_per_epoch=len(train_dataset),
  validation_data=val_dataset, validation_steps=len(val_dataset), epochs=1, verbose=1)


312/312 [==============================] - 93s 295ms/step - loss: 0.0400 - accuracy: 0.9918 - val_loss: 5.1716 - val_accuracy: 0.4969
